In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0727 - mean_squared_error: 0.0727
Loss: 0.07273515313863754 Tiempo transcurrido: 48.69355487823486

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 1ms/step - loss: 0.1053 - mean_squared_error: 0.1053
Loss: 0.10532083362340927 Tiempo transcurrido: 25.29033660888672

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.2044 - mean_squared_error: 0.2044
Loss: 0.20440304279327393 Tiempo transcurrido: 14.45798397064209

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 18.0016 - mean_squared_error: 18.0016
Loss: 18.0015811920166

5/5 [==============================] - 0s 1ms/step - loss: 0.0367 - mean_squared_error: 0.0367
Loss: 0.03668523207306862 Tiempo transcurrido: 10.11312222480774

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0326 - mean_squared_error: 0.0326
Loss: 0.03260987997055054 Tiempo transcurrido: 61.87802529335022

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0399 - mean_squared_error: 0.0399
Loss: 0.039918720722198486 Tiempo transcurrido: 31.485239028930664

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0771 - mean_squared_error: 0.0771
Loss: 0.07709096372127533 Tiempo transcurrido: 17.644587993621826

--- Starting trial: run-35
{'layers': 2, 'num_units':

5/5 [==============================] - 0s 2ms/step - loss: 0.0325 - mean_squared_error: 0.0325
Loss: 0.03246098756790161 Tiempo transcurrido: 27.388060331344604

--- Starting trial: run-63
{'layers': 2, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0483 - mean_squared_error: 0.0483
Loss: 0.04833479970693588 Tiempo transcurrido: 16.505651473999023

--- Starting trial: run-64
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0303 - mean_squared_error: 0.0303
Loss: 0.03031747415661812 Tiempo transcurrido: 67.12253785133362

--- Starting trial: run-65
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0355 - mean_squared_error: 0.0355
Loss: 0.03547797352075577 Tiempo transcurrido: 34.48618817329407

--- Starting trial: run-66
{'layers': 3, 'num_units':

5/5 [==============================] - 0s 2ms/step - loss: 0.0580 - mean_squared_error: 0.0580
Loss: 0.05796166509389877 Tiempo transcurrido: 47.50454926490784

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0388 - mean_squared_error: 0.0388
Loss: 0.03878697380423546 Tiempo transcurrido: 26.438440322875977

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.0451 - mean_squared_error: 0.0451
Loss: 0.045075759291648865 Tiempo transcurrido: 18.38773250579834

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0314 - mean_squared_error: 0.0314
Loss: 0.03136089816689491 Tiempo transcurrido: 78.01866841316223

--- Starting trial: run-97
{'layers': 4, 'num_units':

5/5 [==============================] - 0s 2ms/step - loss: 0.0781 - mean_squared_error: 0.0781
Loss: 0.07814588397741318 Tiempo transcurrido: 120.91579699516296

--- Starting trial: run-125
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0422 - mean_squared_error: 0.0422
Loss: 0.04217260703444481 Tiempo transcurrido: 63.68898582458496

--- Starting trial: run-126
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0424 - mean_squared_error: 0.0424
Loss: 0.04235539957880974 Tiempo transcurrido: 35.023985385894775

--- Starting trial: run-127
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.0572 - mean_squared_error: 0.0572
Loss: 0.05717550590634346 Tiempo transcurrido: 20.931970357894897
128


In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,4,200,0.0001,16,0.032694,21.514782
1,3,150,0.0010,16,0.032786,12.755097


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 8.627 minutes


In [17]:
df["Tiempo de ejecución"]

0     21.514782
1     12.755097
2     15.339474
3     16.255839
4     10.857187
5     27.277918
6     13.625407
7     10.113122
8     22.450776
9     15.340051
10    21.181803
11    10.902846
12    18.257887
13    14.720104
14    16.939221
15    19.023031
16    18.387733
17     9.889611
18    16.505651
19    14.340394
20    17.041763
21    13.327623
22    39.586891
23    20.931970
24    16.439605
25    14.877361
26    18.452252
27     9.621009
28    10.991882
29     9.748454
30    11.533446
31     9.403144
Name: Tiempo de ejecución, dtype: float64